# Trabajo Práctico Integrador 1

## Análisis de datos

#### Integrantes
- Daniel Bazan
- Santiago Casado

### 1. Dataset seleccionado

Conflictos armados en ciudades (Cities and Armed Conflict Events, CACE)

** explicar el dataset **

** info para descargar desde la api

In [61]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, MetaData, text
import json
from io import StringIO

### 2. Descargar el dataset a través de la página oficial

Observando el archivo, se tuvo que agregar el delimitador ";" para poder importarlo correctamente.

In [62]:
path = "dataset/CACE_1989-2017.csv"

# df = pd.read_csv(path, delimiter=';', low_memory=False)
df = pd.read_csv(path, delimiter=';')

df.head()

/var/folders/_f/7gqqb6190c9b0h_7dkhypsj80000gn/T/ipykernel_13230/1640176888.py:4: DtypeWarning: Columns (8,15,17,18,19,20,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=';')


,id,active_year,type_of_violence,conflict_new_id,conflict_name,dyad_new_id,dyad_name,side_a_new_id,gwnoa,side_a,side_b_new_id,gwnob,side_b,where_prec,where_coordinates,City,CACE cityname,Capital,Majorcity,Top3cities,Comment,adm_1,adm_2,latitude,longitude,geom_wkt,priogrid_gid,country,country_id,region,event_clarity,date_prec,year,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,low,high,Unnamed: 42,Unnamed: 43
0,4,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),881,NaN,AQAP,1,Şan‘ā’ city,1,Şan‘ā’,1,1,1,NaN,Am?nat al ‘??imah governorate,NaN,15.354722,44.206667,POINT (44.206667 15.354722),151649,Yemen (North Yemen),678,Middle East,1,1,2010,9/25/2010,9/25/2010,2,0,0,0,2,2,2,NaN,NaN
1,5,1,3,715,Government of Yemen (North Yemen) - Civilians,1182,Government of Yemen (North Yemen) - Civilians,123,678.0,Government of Yemen (North Yemen),1,NaN,Civilians,2,‘Adan town,1,‘Adan,0,0,0,NaN,‘Adan governorate,NaN,12.779444,45.036667,POINT (45.036667 12.779444),148051,Yemen (North Yemen),678,Middle East,1,1,2011,2/19/2011,2/19/2011,0,0,0,0,0,0,2,NaN,NaN
2,6,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),881,NaN,AQAP,4,Abyan governorate,0,NaN,0,0,0,NaN,Abyan governorate,NaN,13.786202,46.141765,POINT (46.141765 13.786202),149493,Yemen (North Yemen),678,Middle East,1,1,2011,4/16/2011,4/16/2011,0,0,0,0,0,0,1,NaN,NaN
3,7,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),881,NaN,AQAP,4,Abyan governorate,0,NaN,0,0,0,NaN,Abyan governorate,NaN,13.786202,46.141765,POINT (46.141765 13.786202),149493,Yemen (North Yemen),678,Middle East,1,1,2012,6/4/2012,6/4/2012,4,1,0,0,5,5,5,NaN,NaN
4,10,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),881,NaN,AQAP,1,‘Ab?dah valley,0,NaN,0,0,0,NaN,Ma'rib governorate,Ma’rib district,15.616667,45.300000,POINT (45.300000 15.616667),152371,Yemen (North Yemen),678,Middle East,1,1,2012,12/9/2012,12/9/2012,0,4,1,0,5,5,5,NaN,NaN


### Preprocesamiento del dataset

1. Se revisan datos faltantes o duplicados

In [63]:
print(f"Valores faltantes: {df.isnull().sum()}")  # Calcular valores faltantes

Valores faltantes: id                        0
active_year               0
type_of_violence          0
conflict_new_id           0
conflict_name             0
dyad_new_id               0
dyad_name                 0
side_a_new_id             0
gwnoa                 32680
side_a                    0
side_b_new_id             0
gwnob                142160
side_b                    0
where_prec                0
where_coordinates         0
City                      0
CACE cityname        117996
Capital                   0
Majorcity                 0
Top3cities                0
Comment              142696
adm_1                  7237
adm_2                 37225
latitude                  0
longitude                 0
geom_wkt                  0
priogrid_gid              0
country                   0
country_id                0
region                    0
event_clarity             0
date_prec                 0
year                      0
date_start                0
date_end                  0
d

In [64]:
print("Cantidad de datos duplicados:", df.duplicated().sum())

Cantidad de datos duplicados: 0


2. Correción de tipo de datos

La libreria Pandas detecta que mas de una columna tiene datos de diferente tipo. Se identifican cada una y se unifica:

In [65]:
mixed_columns = []

i = 0
for col in df.columns:
    unique_types = df[col].apply(type).unique()
    if len(unique_types) > 1:
        mixed_columns.append(col)
        print(f"\n Columna {i}: {col}")
        print("Tipos encontrados:", unique_types)
        
        #Mostrar 2 ejemplos para cada columna con mas de un tipo de dato
        for dtype in unique_types:
            examples = df[df[col].apply(lambda x: type(x) == dtype)][col].head(2).tolist()
            print(f"  - Tipo {dtype}: {examples}")

    i += 1


 Columna 8: gwnoa
Tipos encontrados: [<class 'float'> <class 'str'>]
  - Tipo <class 'float'>: [678.0, 678.0]
  - Tipo <class 'str'>: ['790', '790']

 Columna 15: City
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 1]
  - Tipo <class 'str'>: ['0', '0']

 Columna 16: CACE cityname
Tipos encontrados: [<class 'str'> <class 'float'>]
  - Tipo <class 'str'>: ['Şan‘ā’', '‘Adan ']
  - Tipo <class 'float'>: [nan, nan]

 Columna 17: Capital
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 18: Majorcity
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 19: Top3cities
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 20: Comment
Tipos encontrados: [<class 'float'> <class 'str'>]
  - Tipo <class 'float'>: [nan, nan]
  - Tipo <class 'str'>: ['Not

In [66]:
# Columnas numéricas
import datetime


df['gwnoa'] = pd.to_numeric(df['gwnoa'], errors='coerce').astype('Int64')

# Columnas binarias
binary_cols = ['City', 'Capital', 'Majorcity', 'Top3cities']

for col in binary_cols:
    # Reemplazar valores conocidos
    df[col] = df[col].replace({
        '0': 0, '1': 1,
        0: 0, 1: 1,
        'True': 1, 'False': 0,
        True: 1, False: 0
    })
    
    # Convertir otros valores a NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Rellenar NaN con 0 o 1 según sea apropiado
    df[col] = df[col].fillna(0).astype('int8')

# Columnas de texto
text_cols = ['conflict_name', 'dyad_name', 'side_a', 'side_b', 'where_coordinates',  'CACE cityname', 'Comment', 'adm_1', 'adm_2', 'geom_wkt', 'country', 'region']
for col in text_cols:
    df[col] = df[col].fillna('').astype(str)

# Columnas de fechas
date_cols = ['date_start', 'date_end']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

# Eliminar columnas vacía
df = df.drop(columns=['Unnamed: 43'])
df = df.drop(columns=['Unnamed: 42'])

print("\nTipos unificados:")
print(df.dtypes)


Tipos unificados:
id                            int64
active_year                   int64
type_of_violence              int64
conflict_new_id               int64
conflict_name                object
dyad_new_id                   int64
dyad_name                    object
side_a_new_id                 int64
gwnoa                         Int64
side_a                       object
side_b_new_id                 int64
gwnob                       float64
side_b                       object
where_prec                    int64
where_coordinates            object
City                           int8
CACE cityname                object
Capital                        int8
Majorcity                      int8
Top3cities                     int8
Comment                      object
adm_1                        object
adm_2                        object
latitude                    float64
longitude                   float64
geom_wkt                     object
priogrid_gid                  int64
country  